In [564]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

In [565]:
# Define 5 popular stocks per sector (Yahoo Finance compatible NSE symbols)
tech_stock_list = ['INFY.NS', 'TCS.NS', 'WIPRO.NS', 'HCLTECH.NS', 'TECHM.NS']
fmcg_stock_list = ['HINDUNILVR.NS', 'ITC.NS', 'NESTLEIND.NS', 'BRITANNIA.NS', 'DABUR.NS']
banking_stock_list = ['HDFCBANK.NS', 'ICICIBANK.NS', 'SBIN.NS', 'KOTAKBANK.NS', 'AXISBANK.NS']

# Group into dictionary for flexibility
sector_stocks = {
    'Tech': tech_stock_list,
    'FMCG': fmcg_stock_list,
    'Banking': banking_stock_list
}


In [566]:
def fetch_stock_data(ticker, start_date, end_date):
    print(f"Fetching data for {ticker}...")
    stock = yf.Ticker(ticker)
    data = stock.history(start=start_date, end=end_date)
    
    # Rename columns to standard OHLCV format if needed
    # (yfinance already returns with correct column names)
    data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
    
    # Reset index to make Date a column and convert it safely to string
    data = data.reset_index()
    
    # Convert the Date column to string safely, handling different possible formats
    data['Date'] = data['Date'].astype(str).str.split(' ').str[0]
    data.set_index('Date', inplace=True)
    print(f"Fetching data from {start_date} to {end_date}")
    return data

In [567]:
# Fetch data for each stock in each sector
all_stock_data = {}
end_date = '2025-01-01'
start_date = '2022-01-01'
for sector, stocks in sector_stocks.items():
    all_stock_data[sector] = {}
    for stock in stocks:
        all_stock_data[sector][stock] = fetch_stock_data(stock, start_date, end_date)

Fetching data for INFY.NS...
Fetching data from 2022-01-01 to 2025-01-01
Fetching data for TCS.NS...
Fetching data from 2022-01-01 to 2025-01-01
Fetching data for WIPRO.NS...
Fetching data from 2022-01-01 to 2025-01-01
Fetching data for HCLTECH.NS...
Fetching data from 2022-01-01 to 2025-01-01
Fetching data for TECHM.NS...
Fetching data from 2022-01-01 to 2025-01-01
Fetching data for HINDUNILVR.NS...
Fetching data from 2022-01-01 to 2025-01-01
Fetching data for ITC.NS...
Fetching data from 2022-01-01 to 2025-01-01
Fetching data for NESTLEIND.NS...
Fetching data from 2022-01-01 to 2025-01-01
Fetching data for BRITANNIA.NS...
Fetching data from 2022-01-01 to 2025-01-01
Fetching data for DABUR.NS...
Fetching data from 2022-01-01 to 2025-01-01
Fetching data for HDFCBANK.NS...
Fetching data from 2022-01-01 to 2025-01-01
Fetching data for ICICIBANK.NS...
Fetching data from 2022-01-01 to 2025-01-01
Fetching data for SBIN.NS...
Fetching data from 2022-01-01 to 2025-01-01
Fetching data for KOTA

In [568]:
for stock_name, stock_df in all_stock_data.items():
    print(f"\nStock: {stock_name}")
    print(stock_df)


Stock: Tech
{'INFY.NS':                    Open         High          Low        Close   Volume
Date                                                                   
2022-01-03  1754.770100  1779.217479  1754.770100  1764.716309  3329616
2022-01-04  1764.716308  1772.338738  1745.706925  1765.367065  3921999
2022-01-05  1766.157225  1768.852961  1710.383839  1714.706299  6995719
2022-01-06  1699.229111  1699.229111  1673.201531  1689.747681  6449205
2022-01-07  1687.563162  1706.665592  1679.522590  1686.494263  4834389
...                 ...          ...          ...          ...      ...
2024-12-24  1927.050049  1942.000000  1840.000000  1909.050049  2360544
2024-12-26  1909.050049  1919.750000  1902.300049  1907.400024  3623321
2024-12-27  1909.400024  1924.150024  1903.900024  1916.750000  3937500
2024-12-30  1915.699951  1916.000000  1886.500000  1906.000000  7789055
2024-12-31  1892.300049  1897.000000  1845.050049  1880.000000  3613063

[739 rows x 5 columns], 'TCS.NS':     

In [569]:
from plotly.subplots import make_subplots

import plotly.graph_objects as go

# Create a subplot for each sector
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    subplot_titles=['Tech Sector', 'FMCG Sector', 'Banking Sector']
)

# Plot Tech Sector
for stock in sector_stocks['Tech']:
    fig.add_trace(
        go.Scatter(
            x=all_stock_data['Tech'][stock].index,
            y=all_stock_data['Tech'][stock]['Close'],
            mode='lines',
            name=stock
        ),
        row=1, col=1
    )

# Plot FMCG Sector
for stock in sector_stocks['FMCG']:
    fig.add_trace(
        go.Scatter(
            x=all_stock_data['FMCG'][stock].index,
            y=all_stock_data['FMCG'][stock]['Close'],
            mode='lines',
            name=stock
        ),
        row=2, col=1
    )

# Plot Banking Sector
for stock in sector_stocks['Banking']:
    fig.add_trace(
        go.Scatter(
            x=all_stock_data['Banking'][stock].index,
            y=all_stock_data['Banking'][stock]['Close'],
            mode='lines',
            name=stock
        ),
        row=3, col=1
    )

# Update layout
fig.update_layout(
    height=900,
    title_text='Price Chart of Stocks Sector-wise',
    xaxis_title='Date',
    yaxis_title='Price',
    template='plotly_white'
)

# Show the plot
fig.show()

In [570]:
def select_most_correlated_stocks(all_stock_data, top_n=3):
    selected_stocks = {}

    for sector, stock_data in all_stock_data.items():
        print(f"\nAnalyzing {sector} sector...")
        
        # Get all daily close prices and filter for 2020 to 2021
        prices = pd.DataFrame({
            stock: data['Close'].astype(float) 
            for stock, data in stock_data.items()
        }).loc['2022-01-01':'2023-12-31'].dropna()

        # Calculate daily returns
        returns = prices.pct_change().dropna()

        # Compute correlation matrix
        corr_matrix = returns.corr()

        # Compute average correlation per stock
        avg_corr = corr_matrix.mean().sort_values(ascending=False)

        print(f"Average correlations:\n{avg_corr}")

        # Select top N most correlated stocks
        selected_stocks[sector] = avg_corr.head(top_n).index.tolist()

        print(f"Selected stocks for {sector}: {selected_stocks[sector]}")

    return selected_stocks

# Run the selection
top_stocks_per_sector = select_most_correlated_stocks(all_stock_data)



Analyzing Tech sector...
Average correlations:
INFY.NS       0.754325
TCS.NS        0.746392
HCLTECH.NS    0.732908
WIPRO.NS      0.729539
TECHM.NS      0.722271
dtype: float64
Selected stocks for Tech: ['INFY.NS', 'TCS.NS', 'HCLTECH.NS']

Analyzing FMCG sector...
Average correlations:
HINDUNILVR.NS    0.570288
BRITANNIA.NS     0.541562
NESTLEIND.NS     0.518303
DABUR.NS         0.516026
ITC.NS           0.425045
dtype: float64
Selected stocks for FMCG: ['HINDUNILVR.NS', 'BRITANNIA.NS', 'NESTLEIND.NS']

Analyzing Banking sector...
Average correlations:
ICICIBANK.NS    0.663978
SBIN.NS         0.611677
KOTAKBANK.NS    0.598047
AXISBANK.NS     0.597208
HDFCBANK.NS     0.570386
dtype: float64
Selected stocks for Banking: ['ICICIBANK.NS', 'SBIN.NS', 'KOTAKBANK.NS']


In [571]:
# Re-fetch only the selected stock data to keep things clean
selected_stock_data = {}

for sector, stock_list in top_stocks_per_sector.items():
    selected_stock_data[sector] = {}
    for stock in stock_list:
        selected_stock_data[sector][stock] = all_stock_data[sector][stock]


In [572]:
selected_stock_data

{'Tech': {'INFY.NS':                    Open         High          Low        Close   Volume
  Date                                                                   
  2022-01-03  1754.770100  1779.217479  1754.770100  1764.716309  3329616
  2022-01-04  1764.716308  1772.338738  1745.706925  1765.367065  3921999
  2022-01-05  1766.157225  1768.852961  1710.383839  1714.706299  6995719
  2022-01-06  1699.229111  1699.229111  1673.201531  1689.747681  6449205
  2022-01-07  1687.563162  1706.665592  1679.522590  1686.494263  4834389
  ...                 ...          ...          ...          ...      ...
  2024-12-24  1927.050049  1942.000000  1840.000000  1909.050049  2360544
  2024-12-26  1909.050049  1919.750000  1902.300049  1907.400024  3623321
  2024-12-27  1909.400024  1924.150024  1903.900024  1916.750000  3937500
  2024-12-30  1915.699951  1916.000000  1886.500000  1906.000000  7789055
  2024-12-31  1892.300049  1897.000000  1845.050049  1880.000000  3613063
  
  [739 rows x 5 

In [573]:
def generate_signals(stock_df, short_window=20, long_window=50):
    df = stock_df.copy()
    df['SMA_short'] = df['Close'].rolling(window=short_window).mean()
    df['SMA_long'] = df['Close'].rolling(window=long_window).mean()
    
    # Buy when short SMA crosses above long SMA, Sell when crosses below
    df['Signal'] = 0
    df.loc[df['SMA_short'] > df['SMA_long'], 'Signal'] = 1
    df.loc[df['SMA_short'] < df['SMA_long'], 'Signal'] = -1
    return df[['Close', 'Signal']]


In [574]:
signals_data = {}

for sector, stocks in selected_stock_data.items():
    signals_data[sector] = {}
    for stock, df in stocks.items():
        # Filter data for the first two years (2020 to 2022)
        filtered_df = df.loc['2022-01-01':'2023-12-31']
        signals_data[sector][stock] = generate_signals(filtered_df)


In [575]:
import pandas as pd

# Start with an empty DataFrame for merging
combined_df = None

for sector, stocks in signals_data.items():
    for stock, df in stocks.items():
        # Rename columns to avoid clashes
        renamed = df.rename(columns={
            'Close': f'{stock}_close',
            'Signal': f'{stock}_signal'
        })
        
        if combined_df is None:
            combined_df = renamed
        else:
            combined_df = combined_df.join(renamed, how='outer')

# Forward fill missing prices/signals to maintain alignment
combined_df.fillna(method='ffill', inplace=True)
combined_df.dropna(inplace=True)  # Drop early rows with NA (before any data starts)
combined_df


C:\Users\HP\AppData\Local\Temp\ipykernel_19924\3243973927.py:20: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



,INFY.NS_close,INFY.NS_signal,TCS.NS_close,TCS.NS_signal,HCLTECH.NS_close,HCLTECH.NS_signal,HINDUNILVR.NS_close,HINDUNILVR.NS_signal,BRITANNIA.NS_close,BRITANNIA.NS_signal,NESTLEIND.NS_close,NESTLEIND.NS_signal,ICICIBANK.NS_close,ICICIBANK.NS_signal,SBIN.NS_close,SBIN.NS_signal,KOTAKBANK.NS_close,KOTAKBANK.NS_signal
Date,,,,,,,,,,,,,,,,,,
2022-01-03,1764.716309,0,3576.285400,0,1162.122559,0,2250.232910,0,3451.341064,0,1899.769775,0,741.368286,0,447.229095,0,1819.857544,0
2022-01-04,1765.367065,0,3639.047852,0,1168.519653,0,2288.303711,0,3471.280518,0,1920.150146,0,749.269592,0,459.293274,0,1847.936768,0
2022-01-05,1714.706299,0,3616.753174,0,1148.627319,0,2302.168945,0,3491.554443,0,1924.658691,0,764.005859,0,467.747711,0,1917.311646,0
2022-01-06,1689.747681,0,3566.636963,0,1125.974976,0,2278.916748,0,3506.247070,0,1896.834961,0,761.097351,0,467.082764,0,1887.137817,0
2022-01-07,1686.494263,0,3609.774170,0,1130.925903,0,2302.502930,0,3565.636963,0,1921.482422,0,769.047241,0,466.655273,0,1899.456787,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,1523.653564,1,3752.023682,1,1402.669800,1,2531.527588,1,5094.785645,1,2502.752930,1,985.813293,1,626.248047,1,1858.463135,1
2023-12-26,1505.179443,1,3724.109619,1,1398.306763,1,2540.520996,1,5168.674316,1,2521.852783,1,986.606445,1,627.526550,1,1884.784180,1
2023-12-27,1527.748169,1,3739.464844,1,1411.636353,1,2563.667969,1,5149.770020,1,2537.479492,1,993.695496,1,637.853394,1,1900.267212,1


In [576]:
import pandas as pd
import itertools

# Assuming `combined_df` contains 'close' price columns and 'signal' columns for each stock

# Function to calculate pairwise correlations between stocks
def calculate_pairwise_correlation(combined_df):
    """
    Calculate pairwise correlations between all stock prices.
    """
    # Create an empty dictionary to store pairwise correlation values
    correlation_dict = {}

    # Calculate pairwise correlations for the 'Close' columns
    close_columns = [col for col in combined_df.columns if '_close' in col]
    for stock1 in close_columns:
        for stock2 in close_columns:
            if stock1 != stock2:  # Don't calculate correlation for the same stock
                pair = tuple(sorted([stock1, stock2]))  # Ensure pairs are ordered
                if pair not in correlation_dict:  # Avoid duplicate pairs
                    correlation = combined_df[stock1].corr(combined_df[stock2])
                    correlation_dict[pair] = correlation

    return correlation_dict

# Function to find the best diversified triplet
def find_best_diversified_triplet(combined_df, selected_stock_data):
    """
    Select a triplet of stocks (one from each sector) that minimizes pairwise correlations
    between them.
    """
    # Calculate pairwise correlations for all stocks
    correlation_dict = calculate_pairwise_correlation(combined_df)

    # Create a list of all possible combinations of stocks from different sectors
    sectors = list(selected_stock_data.keys())
    stock_combinations = list(itertools.product(
        selected_stock_data[sectors[0]].keys(),
        selected_stock_data[sectors[1]].keys(),
        selected_stock_data[sectors[2]].keys()
    ))

    # Initialize the variables for the best triplet
    best_triplet = None
    min_total_correlation = float('inf')  # Start with an infinitely large correlation

    # Loop through each combination of stocks and calculate the total correlation
    for combo in stock_combinations:
        stock_A, stock_B, stock_C = combo
        
        # Get the correlation pairs
        correlation_AB = correlation_dict.get(tuple(sorted([f'{stock_A}_close', f'{stock_B}_close'])), 0)
        correlation_AC = correlation_dict.get(tuple(sorted([f'{stock_A}_close', f'{stock_C}_close'])), 0)
        correlation_BC = correlation_dict.get(tuple(sorted([f'{stock_B}_close', f'{stock_C}_close'])), 0)
        
        # Calculate the total correlation for this triplet
        total_correlation = correlation_AB + correlation_AC + correlation_BC
        
        # If this triplet has a lower total correlation, update the best triplet
        if total_correlation < min_total_correlation:
            min_total_correlation = total_correlation
            best_triplet = combo

    print(f"Best triplet of stocks: {best_triplet} with Total Correlation: {min_total_correlation:.4f}")
    return best_triplet

# Example usage:

# Assuming `selected_stock_data` is your data with stocks and their signals
# selected_stock_data = {
#     'Tech': {'TCS': tcs_df, 'INFY': infy_df, 'WIPRO': wipro_df},
#     'FMCG': {'HINDUNILVR': hindunilvr_df, 'ITC': itc_df, 'NESTLEIND': nestleind_df},
#     'Banking': {'HDFCBANK': hdfc_df, 'ICICIBANK': icici_df, 'SBIN': sbi_df}
# }

# Call the function to find the best diversified triplet
best_triplet = find_best_diversified_triplet(combined_df, selected_stock_data)


Best triplet of stocks: ('INFY.NS', 'BRITANNIA.NS', 'KOTAKBANK.NS') with Total Correlation: -0.4573


In [577]:
# Define the stocks (one from each sector) using data from 2022 to 2024
tech_stock = best_triplet[0]       # Tech stock
fmcg_stock = best_triplet[1]       # FMCG stock
banking_stock = best_triplet[2]    # Banking stock

# Filter data for the years 2022 to 2024
start_date = '2024-01-01'
end_date = '2024-12-31'

tech_stock = selected_stock_data['Tech'][tech_stock].loc[start_date:end_date]
fmcg_stock = selected_stock_data['FMCG'][fmcg_stock].loc[start_date:end_date]
banking_stock = selected_stock_data['Banking'][banking_stock].loc[start_date:end_date]

# Display the filtered data
print(f"Tech Stock Data ({tech_stock}):")
print(tech_stock.head())

print(f"\nFMCG Stock Data ({fmcg_stock}):")
print(fmcg_stock.head())

print(f"\nBanking Stock Data ({banking_stock}):")
print(banking_stock.head())

Tech Stock Data (                   Open         High          Low        Close   Volume
Date                                                                   
2024-01-01  1500.353824  1517.901822  1496.697991  1512.393677  2808451
2024-01-02  1507.178071  1510.248994  1484.755629  1495.869385  5396313
2024-01-03  1481.782223  1481.782223  1450.634573  1452.584351  7550363
2024-01-04  1467.158845  1481.587247  1455.411507  1474.714233  7056038
2024-01-05  1468.913619  1501.036155  1468.426174  1494.065796  8895403
...                 ...          ...          ...          ...      ...
2024-12-24  1927.050049  1942.000000  1840.000000  1909.050049  2360544
2024-12-26  1909.050049  1919.750000  1902.300049  1907.400024  3623321
2024-12-27  1909.400024  1924.150024  1903.900024  1916.750000  3937500
2024-12-30  1915.699951  1916.000000  1886.500000  1906.000000  7789055
2024-12-31  1892.300049  1897.000000  1845.050049  1880.000000  3613063

[246 rows x 5 columns]):
                   Op

In [578]:
# # Calculate date range (3 years from today)
# end_date = datetime.now().strftime('%Y-%m-%d')
# start_date = (datetime.now() - timedelta(days=5*365)).strftime('%Y-%m-%d')

# print(f"Fetching data from {start_date} to {end_date}")

# # Function to fetch data for a single stock
# def fetch_stock_data(ticker):
#     print(f"Fetching data for {ticker}...")
#     stock = yf.Ticker(ticker)
#     data = stock.history(start=start_date, end=end_date)
    
#     # Rename columns to standard OHLCV format if needed
#     # (yfinance already returns with correct column names)
#     data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
    
#     # Reset index to make Date a column and convert it safely to string
#     data = data.reset_index()
    
#     # Convert the Date column to string safely, handling different possible formats
#     data['Date'] = data['Date'].astype(str).str.split(' ').str[0]
    
#     return data

# # Fetch data for each sector
# tech_data = fetch_stock_data(tech_stock)
# fmcg_data = fetch_stock_data(fmcg_stock)
# banking_data = fetch_stock_data(banking_stock)

# # Save to CSV files
# tech_data.to_csv(f"stock1_ohlcv.csv", index=False)
# fmcg_data.to_csv(f"stock2_ohlcv.csv", index=False)
# banking_data.to_csv(f"stock3_ohlcv.csv", index=False)

# # Print the first few rows of each dataset to verify
# print(f"\n{tech_stock} (Tech) - First 3 rows:")
# print(tech_data.head(3))

# print(f"\n{fmcg_stock} (FMCG) - First 3 rows:")
# print(fmcg_data.head(3))

# print(f"\n{banking_stock} (Banking) - First 3 rows:")
# print(banking_data.head(3))

# print("\nData saved to CSV files successfully!")

In [579]:
# tech_stock = 
# fmcg_stock = pd.read_csv('ITC.NS_ohlcv.csv', index_col=0)
# banking_stock = pd.read_csv('HDFCBANK.NS_ohlcv.csv', index_col=0)
# # Display the first few rows of each dataset to verify
# print("\nTech Stock Data:")
# print(tech_stock.head())
# print("\nFMCG Stock Data:")
# print(fmcg_stock.head())
# print("\nBanking Stock Data:")
# print(banking_stock.head())


In [580]:
import plotly.graph_objects as go

# Create a figure
fig = go.Figure()

# Add traces for each stock using the index for the x-axis
fig.add_trace(go.Scatter(x=tech_stock.index, y=tech_stock['Close'], mode='lines', name='Tech Stock', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=fmcg_stock.index, y=fmcg_stock['Close'], mode='lines', name='FMCG Stock', line=dict(color='green')))
fig.add_trace(go.Scatter(x=banking_stock.index, y=banking_stock['Close'], mode='lines', name='Banking Stock', line=dict(color='red')))

# Update layout
fig.update_layout(
    title='Stock Prices Over Time',
    xaxis_title='Date',
    yaxis_title='Closing Price',
    legend_title='Stocks',
    xaxis=dict(rangeslider=dict(visible=True), type='date'),
    template='plotly_white'
)

# Show the plot
fig.show()

In [581]:
# Calculate EMA for tech_stock
tech_stock['EMA7'] = tech_stock['Close'].ewm(span=7, adjust=False).mean()
tech_stock['EMA14'] = tech_stock['Close'].ewm(span=14, adjust=False).mean()
tech_stock['EMA21'] = tech_stock['Close'].ewm(span=21, adjust=False).mean()

# Calculate EMA for fmcg_stock
fmcg_stock['EMA7'] = fmcg_stock['Close'].ewm(span=7, adjust=False).mean()
fmcg_stock['EMA14'] = fmcg_stock['Close'].ewm(span=14, adjust=False).mean()
fmcg_stock['EMA21'] = fmcg_stock['Close'].ewm(span=21, adjust=False).mean()

# Calculate EMA for banking_stock
banking_stock['EMA7'] = banking_stock['Close'].ewm(span=7, adjust=False).mean()
banking_stock['EMA14'] = banking_stock['Close'].ewm(span=14, adjust=False).mean()
banking_stock['EMA21'] = banking_stock['Close'].ewm(span=21, adjust=False).mean()

# Display the first few rows of each dataset to verify
print("\nTech Stock with EMAs:")
print(tech_stock.head())

print("\nFMCG Stock with EMAs:")
print(fmcg_stock.head())

print("\nBanking Stock with EMAs:")
print(banking_stock.head())


Tech Stock with EMAs:
                   Open         High          Low        Close   Volume  \
Date                                                                      
2024-01-01  1500.353824  1517.901822  1496.697991  1512.393677  2808451   
2024-01-02  1507.178071  1510.248994  1484.755629  1495.869385  5396313   
2024-01-03  1481.782223  1481.782223  1450.634573  1452.584351  7550363   
2024-01-04  1467.158845  1481.587247  1455.411507  1474.714233  7056038   
2024-01-05  1468.913619  1501.036155  1468.426174  1494.065796  8895403   

                   EMA7        EMA14        EMA21  
Date                                               
2024-01-01  1512.393677  1512.393677  1512.393677  
2024-01-02  1508.262604  1510.190438  1510.891468  
2024-01-03  1494.343040  1502.509626  1505.590821  
2024-01-04  1489.435839  1498.803574  1502.783859  
2024-01-05  1490.593328  1498.171870  1501.991308  

FMCG Stock with EMAs:
                   Open         High          Low        Close  

C:\Users\HP\AppData\Local\Temp\ipykernel_19924\3531349367.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\HP\AppData\Local\Temp\ipykernel_19924\3531349367.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\HP\AppData\Local\Temp\ipykernel_19924\3531349367.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [582]:
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=3, cols=1, 
    shared_xaxes=True,
    subplot_titles=['Tech Stock', 'FMCG Stock', 'Banking Stock']
)

# Tech Stock subplot (Row 1)
fig.add_trace(go.Scatter(x=tech_stock.index, y=tech_stock['Close'], mode='lines', name='Tech Close'), row=1, col=1)
fig.add_trace(go.Scatter(x=tech_stock.index, y=tech_stock['EMA7'], mode='lines', name='Tech EMA7', line=dict(color='rgba(0, 255, 0, 0.5)')), row=1, col=1)
fig.add_trace(go.Scatter(x=tech_stock.index, y=tech_stock['EMA14'], mode='lines', name='Tech EMA14', line=dict(color='rgba(0, 0, 255, 0.5)')), row=1, col=1)
fig.add_trace(go.Scatter(x=tech_stock.index, y=tech_stock['EMA21'], mode='lines', name='Tech EMA21', line=dict(color='rgba(255, 0, 0, 0.5)')), row=1, col=1)

# FMCG Stock subplot (Row 2)
fig.add_trace(go.Scatter(x=fmcg_stock.index, y=fmcg_stock['Close'], mode='lines', name='FMCG Close'), row=2, col=1)
fig.add_trace(go.Scatter(x=fmcg_stock.index, y=fmcg_stock['EMA7'], mode='lines', name='FMCG EMA7', line=dict(color='rgba(0, 255, 0, 0.5)')), row=2, col=1)
fig.add_trace(go.Scatter(x=fmcg_stock.index, y=fmcg_stock['EMA14'], mode='lines', name='FMCG EMA14', line=dict(color='rgba(0, 0, 255, 0.5)')), row=2, col=1)
fig.add_trace(go.Scatter(x=fmcg_stock.index, y=fmcg_stock['EMA21'], mode='lines', name='FMCG EMA21', line=dict(color='rgba(255, 0, 0, 0.5)')), row=2, col=1)

# Banking Stock subplot (Row 3)
fig.add_trace(go.Scatter(x=banking_stock.index, y=banking_stock['Close'], mode='lines', name='Banking Close'), row=3, col=1)
fig.add_trace(go.Scatter(x=banking_stock.index, y=banking_stock['EMA7'], mode='lines', name='Banking EMA7', line=dict(color='rgba(0, 255, 0, 0.5)')), row=3, col=1)
fig.add_trace(go.Scatter(x=banking_stock.index, y=banking_stock['EMA14'], mode='lines', name='Banking EMA14', line=dict(color='rgba(0, 0, 255, 0.5)')), row=3, col=1)
fig.add_trace(go.Scatter(x=banking_stock.index, y=banking_stock['EMA21'], mode='lines', name='Banking EMA21', line=dict(color='rgba(255, 0, 0, 0.5)')), row=3, col=1)

# Update layout
fig.update_layout(
    height=900,
    title_text='Price Chart and Moving Averages for Tech, FMCG and Banking Stocks'
)

fig.show()

In [ ]:
import numpy as np

def calculate_signals(df):
    # If the columns don't exist, create them using defaults
    df['Signal'] = 0
    df['Signal'] = 0
    df['Position'] = 0
    position = 0
    max_value = 0


    # Loop through the dataframe starting at index 22
    for i in range(22, len(df)):
        df.at[df.index[i], 'Position'] = position

        # Buy / Square Short
        if position == 1 and (df['Close'].iloc[i] < 0.95 * max_value):
            df.at[df.index[i], 'Signal'] = -1
            position = 0
            
        elif (((df['Close'].iloc[i] < df['EMA7'].iloc[i] )or (i >= len(df)-1)) and
            position == 1 
           ):   
            df.at[df.index[i], 'Signal'] = -1
            position = 0
            
        elif (df['Close'].iloc[i] > df['EMA7'].iloc[i] and
            position == 0
            ):
            df.at[df.index[i], 'Signal'] = 1 
            position = 1
            max_value = df['Close'].iloc[i]  # Update max_value to current close price
        if position == 1:
            max_value = max(max_value, df['Close'].iloc[i])  # Update max_value if in position
        # Sell / Square Buy
   
    return df

# Apply the signal calculation for all three stocks
tech_stock = calculate_signals(tech_stock)
fmcg_stock = calculate_signals(fmcg_stock)
banking_stock = calculate_signals(banking_stock)

print("Tech Stock Signal Counts:")
print(tech_stock['Signal'].value_counts())

print("\nFMCG Stock Signal Counts:")
print(fmcg_stock['Signal'].value_counts())

print("\nBanking Stock Signal Counts:")
print(banking_stock['Signal'].value_counts())


Tech Stock Signal Counts:
Signal
 0    198
 1     24
-1     24
Name: count, dtype: int64

FMCG Stock Signal Counts:
Signal
 0    192
 1     27
-1     27
Name: count, dtype: int64

Banking Stock Signal Counts:
Signal
 0    183
 1     32
-1     31
Name: count, dtype: int64


C:\Users\HP\AppData\Local\Temp\ipykernel_19924\2762838560.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\HP\AppData\Local\Temp\ipykernel_19924\2762838560.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\HP\AppData\Local\Temp\ipykernel_19924\2762838560.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [584]:
from plotly.subplots import make_subplots

# Create a subplot with 3 rows (one for each stock)
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=False,
    subplot_titles=['Tech Stock', 'FMCG Stock', 'Banking Stock']
)

# Define a list of tuples for convenience: (dataframe, stock name, row)
stocks = [(tech_stock, 'Tech', 1), (fmcg_stock, 'FMCG', 2), (banking_stock, 'Banking', 3)]

for stock_df, name, row in stocks:
    # Determine color for each segment based on Position (ignore first point)
    colors = np.where(stock_df['Position'] == 1, 'green', np.where(stock_df['Position'] == -1, 'red', 'grey'))
    
    # Plot the full close price in grey (for legend purpose)
    fig.add_trace(
        go.Scatter(
            x=stock_df.index, 
            y=stock_df['Close'], 
            mode='lines', 
            name=f'{name} Close Price',
            line=dict(color='grey'),
            showlegend=True
        ),
        row=row, col=1
    )
    
    # Plot individual segments with color based on Position
    for i in range(1, len(stock_df)):
        fig.add_trace(
            go.Scatter(
                x=[stock_df.index[i-1], stock_df.index[i]], 
                y=[stock_df['Close'].iloc[i-1], stock_df['Close'].iloc[i]], 
                mode='lines', 
                line=dict(color=colors[i]),
                showlegend=False
            ),
            row=row, col=1
        )
        
    # Add buy signals: Signal = 1
    buy_signals = stock_df[stock_df['Signal'] == 1]
    fig.add_trace(
        go.Scatter(
            x=buy_signals.index, 
            y=buy_signals['Close'], 
            mode='markers', 
            name=f'{name} Buy Signal',
            marker=dict(color='green', symbol='triangle-up', size=10),
            showlegend=True
        ),
        row=row, col=1
    )
    
    # Add sell signals: Signal = -1
    sell_signals = stock_df[stock_df['Signal'] == -1]
    fig.add_trace(
        go.Scatter(
            x=sell_signals.index, 
            y=sell_signals['Close'], 
            mode='markers', 
            name=f'{name} Sell Signal',
            marker=dict(color='red', symbol='triangle-down', size=10),
            showlegend=True
        ),
        row=row, col=1
    )

# Update layout and display the figure
fig.update_layout(
    height=900,
    title="Stock Prices with Buy/Sell Signals for Tech, FMCG and Banking Stocks"
)

fig.show()

In [585]:
NUM_TRADING_DAYS = 252
NUM_SIMULATIONS = 10000
risk_free_rate = 0.07

In [586]:
def calculate_port_weights(tech_stock, fmcg_stock, banking_stock, starting_date, ending_date, active1, active2, active3):
    # Calculate daily returns
    stock_data = {}
    stock_data['tech'] = tech_stock['Close'][starting_date:ending_date]
    stock_data['fmcg'] = fmcg_stock['Close'][starting_date:ending_date]
    stock_data['banking'] = banking_stock['Close'][starting_date:ending_date]
    stock_prices = pd.DataFrame(stock_data)
    # Calculate log returns
    log_returns = np.log(stock_prices / stock_prices.shift(1))
    log_returns.dropna(inplace=True)  # First row will have NaN value
    
    # Calculate yearly returns
    yearly_returns = log_returns.mean() * NUM_TRADING_DAYS
    
    # Calculate correlation matrix
    # Calculate correlation matrix
    correlation_matrix = stock_prices.corr()

    # Create a heatmap using Plotly
    # import plotly.graph_objects as go

    # fig = go.Figure(data=go.Heatmap(
    #     z=correlation_matrix.values,
    #     x=correlation_matrix.columns,
    #     y=correlation_matrix.index,
    #     colorscale='RdBu',
    #     zmin=-1,
    #     zmax=1,
    #     colorbar=dict(title="Correlation")
    # ))

    # # Update layout
    # fig.update_layout(
    #     title="Stock Correlation Heatmap",
    #     xaxis_title="Stocks",
    #     yaxis_title="Stocks",
    #     template="plotly_white"
    # )

    # # Show the plot
    # fig.show()
    
    portfolio_weights = []
    portfolio_returns = []
    portfolio_risks = []

    for _ in range(NUM_SIMULATIONS):
        w = np.random.random(3)
        w /= np.sum(w)
        portfolio_weights.append(w)
        port_return = np.sum(log_returns.mean()*w)*NUM_TRADING_DAYS
        portfolio_returns.append(port_return)
        port_risk = np.sqrt(np.dot(w.T,np.dot(log_returns.cov()*NUM_TRADING_DAYS,w)))
        portfolio_risks.append(port_risk)
    
    porfolio_weights = np.array(portfolio_weights)
    portfolio_returns = np.array(portfolio_returns)
    portfolio_risks = np.array(portfolio_risks)

    portfolios = pd.DataFrame({"Return": portfolio_returns,"Risk": portfolio_risks,"Sharpe": (portfolio_returns-risk_free_rate)/portfolio_risks})
    
    
    # import plotly.express as px

    # # Create a scatter plot
    # fig = px.scatter(
    #     x=portfolio_risks,
    #     y=portfolio_returns,
    #     color=(portfolio_returns - risk_free_rate) / portfolio_risks,
    #     labels={'x': 'Expected Risk', 'y': 'Expected Return', 'color': 'Sharpe Ratio'},
    #     title='Portfolio Risk vs Return'
    # )

    # # Update layout for better visualization
    # fig.update_layout(
    #     coloraxis_colorbar=dict(title="Sharpe Ratio"),
    #     template="plotly_white"
    # )

    # # Show the plot
    # fig.show()
    
    sharpe_ratios = (portfolio_returns - risk_free_rate) / portfolio_risks

    ind = 0

    for i in range(len(sharpe_ratios)):
        if(sharpe_ratios[i] == np.amax(sharpe_ratios)):
            ind = i
    print("Maximum Sharpe Ratio = ",sharpe_ratios[ind])
    print("Corresponding Return =  ",portfolio_returns[ind])
    print("Corresponding Risk =  ",portfolio_risks[ind])
    
    if (active1 == 0):
        portfolio_weights[ind][0] = 0.0
        x = portfolio_weights[ind][1] / (portfolio_weights[ind][1] + portfolio_weights[ind][2])
        y = portfolio_weights[ind][2] / (portfolio_weights[ind][1] + portfolio_weights[ind][2])
        portfolio_weights[ind][1] = x
        portfolio_weights[ind][2] = y
        if(active2 == 0):
            portfolio_weights[ind][1] = 0.0
            portfolio_weights[ind][2] = 1.0
        elif(active3 == 0):
            portfolio_weights[ind][2] = 0.0
            portfolio_weights[ind][1] = 1.0
    elif (active2 == 0):
        portfolio_weights[ind][1] = 0.0
        x = portfolio_weights[ind][0] / (portfolio_weights[ind][0] + portfolio_weights[ind][2])
        y = portfolio_weights[ind][2] / (portfolio_weights[ind][0] + portfolio_weights[ind][2])
        portfolio_weights[ind][0] = x
        portfolio_weights[ind][2] = y
        if(active3 == 0):
            portfolio_weights[ind][2] = 0.0
            portfolio_weights[ind][0] = 1.0
        elif(active1 == 0):
            portfolio_weights[ind][0] = 0.0
            portfolio_weights[ind][2] = 1.0
    elif (active3 == 0):
        portfolio_weights[ind][2] = 0.0
        x = portfolio_weights[ind][0] / (portfolio_weights[ind][0] + portfolio_weights[ind][1])
        y = portfolio_weights[ind][1] / (portfolio_weights[ind][0] + portfolio_weights[ind][1])
        portfolio_weights[ind][0] = x
        portfolio_weights[ind][1] = y
        if(active1 == 0):
            portfolio_weights[ind][0] = 0.0
            portfolio_weights[ind][1] = 1.0
        elif(active2 == 0):
            portfolio_weights[ind][1] = 0.0
            portfolio_weights[ind][0] = 1.0
        
        
    return portfolio_weights[ind][0], portfolio_weights[ind][1], portfolio_weights[ind][2]


In [587]:
combined_table = pd.DataFrame({
    'Tech_Close': tech_stock['Close'],
    'Tech_Signal': tech_stock['Signal'],
    'FMCG_Close': fmcg_stock['Close'],
    'FMCG_Signal': fmcg_stock['Signal'],
    'Banking_Close': banking_stock['Close'],
    'Banking_Signal': banking_stock['Signal']
})

In [588]:
def backtest(df, initial_balance=1000000):
    import numpy as np
    import pandas as pd
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import plotly.express as px
    from datetime import datetime

    # Initialize portfolio
    balance = initial_balance
    position1 = position2 = position3 = 0
    holding1 = holding2 = holding3 = 0
    entry_price1 = entry_price2 = entry_price3 = 0
    start_date = df.index[0]
    returns = []
    equity_curve = []
    drawdowns = []
    peak = initial_balance
    trade_returns = []
    trades = []
    prices = []
    
    # Track weights and positions over time
    position_history = []
    weight_history = []
    exposure_history = []
    rolling_sharpe = []
    rolling_sortino = []
    rolling_volatility = []
    current_positions = {'Tech': 0, 'FMCG': 0, 'Banking': 0, 'Cash': initial_balance}
    
    # Rolling window for metrics
    window_size = min(60, len(df) // 4)  # ~3 months or 1/4 of data, whichever is smaller

    for index, row in df.iterrows():
        end_date = index
        signal1 = row['Tech_Signal']
        signal2 = row['FMCG_Signal']
        signal3 = row['Banking_Signal']
        price1 = row['Tech_Close']
        price2 = row['FMCG_Close']
        price3 = row['Banking_Close']
        prices.append((price1, price2, price3))

        # Portfolio value before trades
        portfolio_value = balance + (holding1 * price1) + (holding2 * price2) + (holding3 * price3)
        start_date = (pd.to_datetime(end_date) - pd.DateOffset(years=1)).strftime('%Y-%m-%d')
        # === Tech ===
        if signal1 == 1 and position1 == 0:
            w1, _, _ = calculate_port_weights(tech_stock, fmcg_stock, banking_stock, start_date, end_date, 1, int(position2 != 0), int(position3 != 0))
            holding1 = (w1 * balance) / price1
            balance -= holding1 * price1
            entry_price1 = price1
            position1 = 1
            trades.append(('Buy', price1, index, 'Tech', 0.0))

        elif signal1 == -1 and position1 == 1:
            balance += holding1 * price1
            ret = (price1 - entry_price1) / entry_price1
            trade_returns.append(ret)
            trades.append(('Sell', price1, index, 'Tech', ret))
            holding1 = 0
            position1 = 0

        # === FMCG ===
        elif signal2 == 1 and position2 == 0:
            _, w2, _ = calculate_port_weights(tech_stock, fmcg_stock, banking_stock, start_date, end_date, int(position1 != 0), 1, int(position3 != 0))
            holding2 = (w2 * balance) / price2
            balance -= holding2 * price2
            entry_price2 = price2
            position2 = 1
            trades.append(('Buy', price2, index, 'FMCG', 0.0))

        elif signal2 == -1 and position2 == 1:
            balance += holding2 * price2
            ret = (price2 - entry_price2) / entry_price2
            trade_returns.append(ret)
            trades.append(('Sell', price2, index, 'FMCG', ret))
            holding2 = 0
            position2 = 0

        # === Banking ===
        elif signal3 == 1 and position3 == 0:
            _, _, w3 = calculate_port_weights(tech_stock, fmcg_stock, banking_stock, start_date, end_date, int(position1 != 0), int(position2 != 0), 1)
            holding3 = (w3 * balance) / price3
            balance -= holding3 * price3
            entry_price3 = price3
            position3 = 1
            trades.append(('Buy', price3, index, 'Banking', 0.0))

        elif signal3 == -1 and position3 == 1:
            balance += holding3 * price3
            ret = (price3 - entry_price3) / entry_price3
            trade_returns.append(ret)
            trades.append(('Sell', price3, index, 'Banking', ret))
            holding3 = 0
            position3 = 0

        # Portfolio value after trades
        portfolio_value = balance + (holding1 * price1) + (holding2 * price2) + (holding3 * price3)
        equity_curve.append(portfolio_value)

        # Calculate daily return
        if len(equity_curve) > 1:
            period_return = (portfolio_value / equity_curve[-2]) - 1
        else:
            period_return = (portfolio_value / initial_balance) - 1
        returns.append(period_return)

        # Track maximum drawdown
        if portfolio_value > peak:
            peak = portfolio_value
        drawdowns.append((portfolio_value - peak) / peak)
        
        # Calculate current positions and weights
        tech_value = holding1 * price1
        fmcg_value = holding2 * price2
        banking_value = holding3 * price3
        total_value = tech_value + fmcg_value + banking_value + balance
        
        current_positions = {
            'Tech': tech_value,
            'FMCG': fmcg_value,
            'Banking': banking_value,
            'Cash': balance
        }
        
        # Record position history
        position_history.append(current_positions.copy())
        
        # Calculate weights
        weights = {
            'Tech': tech_value / total_value if total_value > 0 else 0,
            'FMCG': fmcg_value / total_value if total_value > 0 else 0,
            'Banking': banking_value / total_value if total_value > 0 else 0,
            'Cash': balance / total_value if total_value > 0 else 1
        }
        weight_history.append(weights)
        
        # Calculate total exposure
        exposure = (tech_value + fmcg_value + banking_value) / total_value if total_value > 0 else 0
        exposure_history.append(exposure)
        
        # Calculate rolling metrics (if we have enough data)
        if len(returns) >= window_size:
            window_returns = returns[-window_size:]
            rolling_vol = np.std(window_returns) * np.sqrt(252)
            rolling_volatility.append(rolling_vol)
            
            window_avg_return = np.mean(window_returns)
            if rolling_vol > 0:
                rolling_sharpe.append(window_avg_return / rolling_vol * np.sqrt(252))
            else:
                rolling_sharpe.append(0)
                
            negative_returns = [r for r in window_returns if r < 0]
            downside_risk = np.std(negative_returns) * np.sqrt(252) if negative_returns else 0.0001
            rolling_sortino.append(window_avg_return / downside_risk * np.sqrt(252))
        else:
            # Fill with zeros until we have enough data
            rolling_volatility.append(0)
            rolling_sharpe.append(0)
            rolling_sortino.append(0)

    # Add to DataFrame
    df['Equity Curve'] = equity_curve
    df['Returns'] = returns
    df['Drawdowns'] = drawdowns
    df['Cumulative Return'] = (1 + df['Returns']).cumprod()
    df['Rolling Sharpe'] = [0] * (len(df) - len(rolling_sharpe)) + rolling_sharpe
    df['Rolling Sortino'] = [0] * (len(df) - len(rolling_sortino)) + rolling_sortino
    df['Rolling Volatility'] = [0] * (len(df) - len(rolling_volatility)) + rolling_volatility
    df['Market Exposure'] = exposure_history

    # Convert position history to DataFrame
    position_df = pd.DataFrame(position_history, index=df.index)
    weight_df = pd.DataFrame(weight_history, index=df.index)
    
    # Performance metrics
    total_return = (equity_curve[-1] / initial_balance) - 1
    years = len(df) / 252
    cagr = (equity_curve[-1] / initial_balance) ** (1 / years) - 1 if years > 0 else 0
    max_drawdown = min(drawdowns)
    volatility = np.std(returns) * np.sqrt(252)
    sharpe_ratio = np.mean(returns) / np.std(returns) * np.sqrt(252) if np.std(returns) > 0 else np.nan
    negative_returns = [r for r in returns if r < 0]
    sortino_ratio = np.mean(returns) / np.std(negative_returns) * np.sqrt(252) if negative_returns else np.nan
    win_rate = len([r for r in trade_returns if r > 0]) / len(trade_returns) if trade_returns else np.nan
    
    # Additional metrics
    avg_profit = np.mean([r for r in trade_returns if r > 0]) if [r for r in trade_returns if r > 0] else 0
    avg_loss = np.mean([r for r in trade_returns if r < 0]) if [r for r in trade_returns if r < 0] else 0
    profit_factor = abs(sum([r for r in trade_returns if r > 0]) / sum([r for r in trade_returns if r < 0])) if sum([r for r in trade_returns if r < 0]) else np.inf
    max_consec_losses = 0
    current_streak = 0
    for r in trade_returns:
        if r < 0:
            current_streak += 1
            max_consec_losses = max(max_consec_losses, current_streak)
        else:
            current_streak = 0
            
    # Calculate average and max Sharpe ratio
    avg_sharpe = np.mean([s for s in rolling_sharpe if s > 0])
    max_sharpe = np.max(rolling_sharpe) if rolling_sharpe else 0
    max_sharpe_date = df.index[np.argmax(rolling_sharpe)] if rolling_sharpe else None

    # Fix Benchmark Return
    initial_benchmark = sum(prices[0]) / 3
    final_benchmark = sum(prices[-1]) / 3
    benchmark_return = (final_benchmark - initial_benchmark) / initial_benchmark

    # Trade analysis by asset
    trade_df = pd.DataFrame(trades, columns=['Type', 'Price', 'Date', 'Asset', 'Return'])
    trade_summary = trade_df[trade_df['Type'] == 'Sell'].groupby('Asset')['Return'].agg(['count', 'mean', 'sum', 'std'])
    trade_summary['win_rate'] = trade_df[trade_df['Type'] == 'Sell'].groupby('Asset')['Return'].apply(lambda x: (x > 0).mean())

    # Print results
    print('\n=== BACKTEST RESULTS ===')
    print(f'Total Return: {total_return:.2%}')
    print(f'Benchmark Return: {benchmark_return:.2%}')
    print(f'CAGR: {cagr:.2%}')
    print(f'Max Drawdown: {max_drawdown:.2%}')
    print(f'Sharpe Ratio: {sharpe_ratio:.2f}')
    print(f'Sortino Ratio: {sortino_ratio:.2f}')
    print(f'Annualized Volatility: {volatility:.2%}')
    print(f'Win Rate: {win_rate:.2%}')
    print(f'Number of Trades: {len(trades)}')
    print(f'Average Profit: {avg_profit:.2%}')
    print(f'Average Loss: {avg_loss:.2%}')
    print(f'Profit Factor: {profit_factor:.2f}')
    print(f'Max Consecutive Losses: {max_consec_losses}')
    print(f'Max Sharpe Ratio: {max_sharpe:.2f} (on {max_sharpe_date})')
    
    print('\n=== TRADE SUMMARY BY ASSET ===')
    print(trade_summary)

    # Optional export
    df.to_csv('backtest_equity_curve.csv')
    position_df.to_csv('position_history.csv')
    weight_df.to_csv('weight_history.csv')
    pd.DataFrame(trades, columns=['Type', 'Price', 'Date', 'Asset', 'Return']).to_csv('trade_log.csv')

    # =============== VISUALIZATIONS ===============

    # PLOT 1: Main Performance Dashboard
    fig1 = make_subplots(rows=3, cols=1,
                        subplot_titles=('Portfolio Value Over Time', 'Drawdowns Over Time', 'Average Close Price Over Time'),
                        vertical_spacing=0.12,
                        row_heights=[0.7, 0.3, 0.7])

    fig1.add_trace(go.Scatter(x=df.index, y=equity_curve, mode='lines', name='Portfolio Value', line=dict(color='black', width=2)), row=1, col=1)
    fig1.add_trace(go.Scatter(x=df.index, y=drawdowns, mode='lines', name='Drawdown', fill='tozeroy', line=dict(color='red', width=1), fillcolor='rgba(255, 0, 0, 0.2)'), row=2, col=1)
    
    avg_prices = [sum(p) / 3 for p in prices]
    fig1.add_trace(go.Scatter(x=df.index, y=avg_prices, mode='lines', name='Avg Close Price', line=dict(color='royalblue', width=2)), row=3, col=1)

    # Buy/Sell markers
    trade_df = pd.DataFrame(trades, columns=['Type', 'Price', 'Date', 'Asset', 'Return'])
    for action, color, symbol in [('Buy', 'green', 'triangle-up'), ('Sell', 'red', 'triangle-down')]:
        filtered = trade_df[trade_df['Type'] == action]
        if not filtered.empty:
            fig1.add_trace(
                go.Scatter(
                    x=filtered['Date'],
                    y=[equity_curve[df.index.get_loc(d)] for d in filtered['Date']],
                    mode='markers',
                    name=action,
                    marker=dict(color=color, size=10, symbol=symbol)
                ),
                row=1, col=1
            )
            fig1.add_trace(
                go.Scatter(
                    x=filtered['Date'],
                    y=[avg_prices[df.index.get_loc(d)] for d in filtered['Date']],
                    mode='markers',
                    name=action,
                    marker=dict(color=color, size=10, symbol=symbol)
                ),
                row=3, col=1
            )

    # Performance annotation
    metrics_text = (
        f'Total Return: {total_return:.2%}<br>'
        f'CAGR: {cagr:.2%}<br>'
        f'Max Drawdown: {max_drawdown:.2%}<br>'
        f'Sharpe Ratio: {sharpe_ratio:.2f}<br>'
        f'Sortino Ratio: {sortino_ratio:.2f}<br>'
        f'Volatility: {volatility:.2%}<br>'
        f'Win Rate: {win_rate:.2%}<br>'
        f'Number of Trades: {len(trades)}'
    )

    fig1.add_annotation(
        xref='paper', yref='paper', x=1.0, y=1.0,
        text=metrics_text, showarrow=False, align='right',
        font=dict(size=10), bgcolor='rgba(255,255,255,0.8)',
        bordercolor='black', borderwidth=1, borderpad=4,
        xanchor='right', yanchor='top'
    )

    fig1.update_layout(
        height=800, width=1200, title_text='Trading Strategy Backtest Results', title_x=0.5,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='center', x=0.5),
        hovermode='x unified', template='plotly_white',
        margin=dict(l=50, r=50, t=80, b=50)
    )

    fig1.update_xaxes(title_text='Date', row=2, col=1)
    fig1.update_yaxes(title_text='Portfolio Value', row=1, col=1)
    fig1.update_yaxes(title_text='Drawdown', tickformat='.1%', row=2, col=1)

    # PLOT 2: Portfolio Weights & Allocation
    fig2 = make_subplots(rows=2, cols=1, 
                         subplot_titles=('Portfolio Allocation Over Time', 'Portfolio Weight Distribution'),
                         vertical_spacing=0.15,
                         row_heights=[0.7, 0.3])

    # Portfolio allocation stacked area chart
    fig2.add_trace(
        go.Scatter(x=df.index, y=position_df['Tech'], mode='lines', name='Tech', 
                  fill='tozeroy', line=dict(width=0), fillcolor='rgba(99, 110, 250, 0.5)'),
        row=1, col=1
    )
    fig2.add_trace(
        go.Scatter(x=df.index, y=position_df['Tech'] + position_df['FMCG'], mode='lines', name='FMCG',
                  fill='tonexty', line=dict(width=0), fillcolor='rgba(239, 85, 59, 0.5)'),
        row=1, col=1
    )
    fig2.add_trace(
        go.Scatter(x=df.index, y=position_df['Tech'] + position_df['FMCG'] + position_df['Banking'], 
                  mode='lines', name='Banking', fill='tonexty', line=dict(width=0), 
                  fillcolor='rgba(0, 204, 150, 0.5)'),
        row=1, col=1
    )
    fig2.add_trace(
        go.Scatter(x=df.index, y=equity_curve, mode='lines', name='Cash',
                  fill='tonexty', line=dict(width=0), fillcolor='rgba(171, 171, 171, 0.5)'),
        row=1, col=1
    )

    # Portfolio weights line chart
    fig2.add_trace(
        go.Scatter(x=df.index, y=weight_df['Tech'], mode='lines', name='Tech Weight', 
                  line=dict(color='rgb(99, 110, 250)', width=2)),
        row=2, col=1
    )
    fig2.add_trace(
        go.Scatter(x=df.index, y=weight_df['FMCG'], mode='lines', name='FMCG Weight',
                  line=dict(color='rgb(239, 85, 59)', width=2)),
        row=2, col=1
    )
    fig2.add_trace(
        go.Scatter(x=df.index, y=weight_df['Banking'], mode='lines', name='Banking Weight',
                  line=dict(color='rgb(0, 204, 150)', width=2)),
        row=2, col=1
    )
    fig2.add_trace(
        go.Scatter(x=df.index, y=weight_df['Cash'], mode='lines', name='Cash Weight',
                  line=dict(color='rgb(171, 171, 171)', width=2)),
        row=2, col=1
    )

    fig2.update_layout(
        height=700, width=1200, title_text='Portfolio Allocation & Weight Distribution', 
        title_x=0.5, legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='center', x=0.5),
        hovermode='x unified', template='plotly_white'
    )
    fig2.update_yaxes(title_text='Allocation ($)', row=1, col=1)
    fig2.update_yaxes(title_text='Weight (%)', row=2, col=1, tickformat='.1%')

    # PLOT 3: Risk Metrics
    fig3 = make_subplots(rows=2, cols=2,
                        subplot_titles=('Rolling Sharpe Ratio', 'Rolling Sortino Ratio', 
                                        'Rolling Volatility', 'Market Exposure'),
                        vertical_spacing=0.15,
                        horizontal_spacing=0.1)

    # Rolling Sharpe
    fig3.add_trace(
        go.Scatter(x=df.index, y=df['Rolling Sharpe'], mode='lines', name='Rolling Sharpe',
                  line=dict(color='rgb(99, 110, 250)', width=2)),
        row=1, col=1
    )
    # Mark maximum Sharpe point
    if max_sharpe_date:
        fig3.add_trace(
            go.Scatter(x=[max_sharpe_date], y=[max_sharpe], mode='markers', name='Max Sharpe',
                      marker=dict(color='darkblue', size=10, symbol='star')),
            row=1, col=1
        )

    # Rolling Sortino
    fig3.add_trace(
        go.Scatter(x=df.index, y=df['Rolling Sortino'], mode='lines', name='Rolling Sortino',
                  line=dict(color='rgb(239, 85, 59)', width=2)),
        row=1, col=2
    )

    # Rolling Volatility
    fig3.add_trace(
        go.Scatter(x=df.index, y=df['Rolling Volatility'], mode='lines', name='Rolling Volatility',
                  line=dict(color='rgb(0, 204, 150)', width=2)),
        row=2, col=1
    )

    # Market Exposure
    fig3.add_trace(
        go.Scatter(x=df.index, y=df['Market Exposure'], mode='lines', name='Market Exposure',
                  line=dict(color='rgb(171, 171, 171)', width=2)),
        row=2, col=2
    )

    fig3.update_layout(
        height=700, width=1200, title_text='Risk Metrics Over Time', title_x=0.5,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='center', x=0.5),
        hovermode='x unified', template='plotly_white'
    )
    fig3.update_yaxes(tickformat='.1%', row=2, col=1)
    fig3.update_yaxes(tickformat='.1%', row=2, col=2)

    # PLOT 4: Trade Analysis
    fig4 = make_subplots(rows=1, cols=2,
                        subplot_titles=('Trade Returns Distribution', 'Cumulative Trade Profit/Loss'),
                        horizontal_spacing=0.1)

    # Trade Returns Histogram
    fig4.add_trace(
        go.Histogram(x=[r for r in trade_returns if r > 0], name='Winning Trades',
                    marker_color='rgba(0, 204, 150, 0.7)', autobinx=True),
        row=1, col=1
    )
    fig4.add_trace(
        go.Histogram(x=[r for r in trade_returns if r <= 0], name='Losing Trades',
                    marker_color='rgba(239, 85, 59, 0.7)', autobinx=True),
        row=1, col=1
    )

    # Cumulative P&L
    trade_pnl = pd.DataFrame(trades, columns=['Type', 'Price', 'Date', 'Asset', 'Return'])
    trade_pnl = trade_pnl[trade_pnl['Type'] == 'Sell']
    trade_pnl['Cumulative Return'] = trade_pnl['Return'].cumsum()

    fig4.add_trace(
        go.Scatter(x=trade_pnl['Date'], y=trade_pnl['Cumulative Return'], mode='lines', name='Cumulative P&L',
                  line=dict(color='black', width=2)),
        row=1, col=2
    )

    # Add markers for each trade
    fig4.add_trace(
        go.Scatter(x=trade_pnl['Date'], y=trade_pnl['Return'], mode='markers', name='Individual Trades',
                  marker=dict(color=trade_pnl['Return'].apply(lambda x: 'green' if x > 0 else 'red'), 
                             size=8, symbol='circle')),
        row=1, col=2
    )

    fig4.update_layout(
        height=500, width=1200, title_text='Trade Analysis', title_x=0.5,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='center', x=0.5),
        hovermode='x unified', template='plotly_white'
    )
    fig4.update_xaxes(title_text='Return (%)', row=1, col=1)
    fig4.update_yaxes(title_text='Count', row=1, col=1)
    fig4.update_yaxes(title_text='Cumulative Return', row=1, col=2, tickformat='.1%')

    # PLOT 5: Asset Performance Comparison
    fig5 = go.Figure()

    # Add traces for each asset
    for asset, color in zip(['Tech', 'FMCG', 'Banking'], ['blue', 'red', 'green']):
        asset_trades = trade_df[(trade_df['Type'] == 'Sell') & (trade_df['Asset'] == asset)]
        if not asset_trades.empty:
            fig5.add_trace(
                go.Box(y=asset_trades['Return'], name=asset, marker_color=color,
                      boxmean=True, jitter=0.3, pointpos=-1.8)
            )

    fig5.update_layout(
        height=500, width=1200, title_text='Asset Performance Comparison', title_x=0.5,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='center', x=0.5),
        hovermode='closest', template='plotly_white',
        yaxis=dict(title='Return per Trade', tickformat='.1%')
    )

    # Display all plots
    print("\nDisplaying Performance Dashboard...")
    fig1.show()
    print("\nDisplaying Portfolio Allocation...")
    fig2.show()
    print("\nDisplaying Risk Metrics...")
    fig3.show()
    print("\nDisplaying Trade Analysis...")
    fig4.show()
    print("\nDisplaying Asset Performance Comparison...")
    fig5.show()
    
    # Return dataframes and figures
    return df, trades, {
        'main_dashboard': fig1,
        'portfolio_allocation': fig2,
        'risk_metrics': fig3,
        'trade_analysis': fig4,
        'asset_comparison': fig5
    }

In [589]:
combined_table.head()

,Tech_Close,Tech_Signal,FMCG_Close,FMCG_Signal,Banking_Close,Banking_Signal
Date,,,,,,
2024-01-01,1512.393677,0,5251.446289,0,1907.059814,0
2024-01-02,1495.869385,0,5220.894531,0,1863.008179,0
2024-01-03,1452.584351,0,5186.294434,0,1863.857178,0
2024-01-04,1474.714233,0,5292.413086,0,1861.559692,0
2024-01-05,1494.065796,0,5204.606445,0,1845.577148,0


In [ ]:
df2, trades, fig = backtest(combined_table, initial_balance=1000000)

Maximum Sharpe Ratio =  2.805020793987058
Corresponding Return =   1.0022416448980376
Corresponding Risk =   0.3323474987766307
Maximum Sharpe Ratio =  2.06011084360802
Corresponding Return =   0.6995357734494828
Corresponding Risk =   0.30558344731923825
Maximum Sharpe Ratio =  2.1708980769812305
Corresponding Return =   0.7103435544147193
Corresponding Risk =   0.2949671203841855
Maximum Sharpe Ratio =  1.6453340182896112
Corresponding Return =   0.536288056632636
Corresponding Risk =   0.28340024058905716
Maximum Sharpe Ratio =  1.3614846888296543
Corresponding Return =   0.4263509529638522
Corresponding Risk =   0.2617370256805275
Maximum Sharpe Ratio =  0.5786752865522361
Corresponding Return =   0.22333986959811256
Corresponding Risk =   0.26498430667692047
Maximum Sharpe Ratio =  0.5416117541475701
Corresponding Return =   0.21124794954508577
Corresponding Risk =   0.26079188360192174
Maximum Sharpe Ratio =  0.47129486520234953
Corresponding Return =   0.19141640758648287
Corres